# Start

In [2]:
import numpy as np
import pandas as pd

In [3]:
def generate_population():
    pass

In [6]:
def get_had_dataframe(
    a: np.array,
    b: np.array,
    v_a: str = 'A',
    v_b: str = 'B',
    feature: str = 'value'
) -> pd.DataFrame:
    a = pd.Series(a)
    b = pd.Series(b)
    size_nonnull_a = a.count()
    size_nonnull_b = b.count()

    if all((not size_nonnull_a, not size_nonnull_b)):
        raise ValueError('Both arrays are empty.')
    if not size_nonnull_a:
        raise ValueError('Array A is empty.')
    if not size_nonnull_b:
        raise ValueError('Array B is empty.')

    df_a = pd.DataFrame(a, columns=[feature])
    df_b = pd.DataFrame(b, columns=[feature])

    df_a = df_a.groupby(feature, as_index=False)\
        .size()\
        .rename(columns={'size': f'count_{v_a}'})
    df_a[f'bin_fraction_{v_a}'] = df_a[f'count_{v_a}'] / size_nonnull_a

    df_b = df_b.groupby(feature, as_index=False)\
        .size()\
        .rename(columns={'size': f'count_{v_b}'})
    df_b[f'bin_fraction_{v_b}'] = df_b[f'count_{v_b}'] / size_nonnull_b

    df_had = df_a.merge(df_b, how='outer', on=feature).fillna(0)
    df_had['diff'] = df_had[f'bin_fraction_{v_b}'] - df_had[f'bin_fraction_{v_a}']
    df_had['half_abs_diff'] = df_had['diff'].abs() / 2
    return df_had


In [ ]:
# a = ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'd']
# b = ['a', 'a', 'a', 'a', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'd', 'd']
# a = [None]
# b = [None]
# a = [1, 1, 1]
# b = [None, None, 1, 2]
# a = np.ones(1000)
# b = np.concatenate([np.ones(500), np.ones(500) * 2])

In [ ]:
a = [1, 2, 3, 1, 2, 3, 1, 2, 3]
b = [1, 1, 3, 2, 2, 3, 3, 2, 3]
a = np.random.randint(0, 10, 10000000)
b = np.random.randint(0, 10, 10000000)
a = np.tile(np.arange(1000), 1000)
b = np.concatenate([a, [1001]])

## Half

In [98]:
n_bins = 9
n_samples_per_bin = 2
a = np.array([[j for i in range(n_samples_per_bin)] for j in range(n_bins)]).flatten()
b = np.array([[n_bins, j] for j in range(n_bins)]).flatten()
print('a:', a)
print('b:', b)


a: [0 0 1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8]
b: [9 0 9 1 9 2 9 3 9 4 9 5 9 6 9 7 9 8]


In [99]:

df_had = get_had_dataframe(a, b)
# df_had.sum().to_frame()

epsilon = 10**-12
df_had['ratio'] = df_had[f'bin_fraction_B'] / (df_had[f'bin_fraction_A'] + epsilon)
df_had['psi'] = 100 * df_had['diff'] * np.log(df_had['ratio'] + epsilon)
(df_had.sum(numeric_only=True).to_frame() * 1000000).astype(int) / 1000000

,0
value,45.000000
count_A,18.000000
bin_fraction_A,1.000000
count_B,18.000000
bin_fraction_B,1.000000
diff,0.000000
half_abs_diff,0.500000
ratio,-2147.483648
psi,1381.551055


## 10%

In [130]:
a = np.concatenate(
    [
        np.ones(1) * 1,
        np.ones(2) * 2,
        np.ones(3) * 3,
        np.ones(4) * 4,
    ]
)
b = np.concatenate(
    [
        np.ones(1) * 1,
        np.ones(2) * 2,
        np.ones(3) * 4,
        np.ones(4) * 3,
    ]
)

(get_had_dataframe(a, b).sum().to_frame() * 100).astype(int) / 100

,0
value,10.0
count_A,10.0
bin_fraction_A,1.0
count_B,10.0
bin_fraction_B,1.0
diff,0.0
half_abs_diff,0.1


# End